In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
# create a new webbrowser session
driver = webdriver.Chrome('G:\\My Drive\\Cs\\2019_01\\SocialNetworkAnalysis\\chromedriver for chrome78\\chromedriver')
driver.implicitly_wait(30)
driver.maximize_window()

In [ ]:
from bs4 import BeautifulSoup 
import json
import time
from datetime import datetime
import pandas as pd
import re

def find_all_comment(pageNameOrId,postId):
    fetch_url = 'https://www.facebook.com/pg/%s/posts/%s/' % (pageNameOrId,postId) 
    driver.get(fetch_url)
    l = 0
    last_l = -1
    comment_list = []

    userIds = ''
    commentIds = ''
    commentTimes = ''
    commentText = ''      
    commentInteractionCount = 0

    dataa = driver.find_elements_by_class_name("_3w53") #all comment
    data = dataa[0]
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    divs = soup.find_all('div', {'class': '_42ef'}) #one comment   
    for div in divs[1:]:
        for div_content in div.find_all('div',{'class':'_6c7i'}):
            comment_text = div.find_all('span',{'class' : '_3l3x'})
            user_id = div.find_all('a', {'class': '_6qw4'})
            comment_interaction = div.find_all('div',{'class' : '_6cuq'})

            commentText =  [span.getText() for span in comment_text] or '-'            
            commentInteractionCount = [int(re.findall("[0-9]*", span.getText())[0]) for span in comment_interaction] or '0'
            userIds = [a['href'][1:] for a in user_id]

        for ul_interact in div.find_all('ul',{'class':'_6coi _6qw9'}) :
            comment_id = div.find_all('a', {'class': '_6qw7'})
            commentIds = [((a['href']).split('?')[1]).split('=')[1] for a in comment_id]
            commentTimes = [int(a.abbr['data-utime']) for a in comment_id] #attribute
            commentTimes = pd.to_datetime(commentTimes[0],unit='s').strftime("%Y-%m-%d %H:%M:%S")

        #print('loop')
        comment_list.append({
            'post_id' : postId,
            'comment_id': commentIds,
            'comment_time': commentTimes,
            'comment_content' : commentText,      
            'user_id': userIds,
            'comment_like_count' : commentInteractionCount,
            'crawl_timestamp' : datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }) 

    #detail = ""
    print(comment_list)
    #print(len(comment_list), 'data were collected.')
    return comment_list

In [ ]:
import random
import os.path
import pandas as pd
import numpy as np
from pandas import read_csv

data = read_csv('khaosod.csv', header=0)
list_post_page_id = ((data['post_page_id'].tolist()))
list_post_id = ((data['post_id'].tolist()))

output_filename = 'postcomment.csv'

for i in range(len(list_post_page_id)):
    print(i)
    pageNameOrId = list_post_page_id[i]
    postId = list_post_id[i]
    count=0
    comment_list = find_all_comment(pageNameOrId,postId)
    #print(comment_list) 

    file = open(output_filename,'a',encoding='utf-8')
    for c in comment_list:       
        file.write(str(c['post_id'])+','+c['comment_id'][0]+','+c['comment_time']+','+c['comment_content'][0]+','+c['user_id'][0]+','+str(c['comment_like_count'][0])+','+c['crawl_timestamp'] )#+','+c[2]+','+c[3])
        file.write('\n')
        count = count+1
        print("write record {}".format(count))
    file.close()